# Survey-to-survey Imputation Analysis


In [40]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score
from sklearn.model_selection import train_test_split

In [41]:
MAIN_DIR = "C:\\Users\\md82\\OneDrive - Anglia Ruskin University\\Documents\\mj-datalab\\poverty-prediction-challenge"
DATA_DIR = MAIN_DIR + "\\data\\raw"
SUB_DIR = MAIN_DIR + "\\data\\submission"
TRAIN_DATA = DATA_DIR + "\\train_hh_features.csv"
TEST_DATA = DATA_DIR + "\\test_hh_features.csv"
HH_DATA = DATA_DIR + "\\train_hh_gt.csv"
PR_DATA = DATA_DIR + "\\train_rates_gt.csv"

train_data = pd.read_csv(TRAIN_DATA)
test_data = pd.read_csv(TEST_DATA)
hh_data = pd.read_csv(HH_DATA)  
pr_data = pd.read_csv(PR_DATA)

In [42]:
# Check dimensions of training features
print("Shape:", train_data.shape)  # (rows, columns)
print("Rows:", len(train_data))
print("Columns:", len(train_data.columns))
train_data.head()

Shape: (104234, 88)
Rows: 104234
Columns: 88


,hhid,com,weight,strata,utl_exp_ppp17,male,hsize,num_children5,num_children10,num_children18,...,consumed4200,consumed4300,consumed4400,consumed4500,consumed4600,consumed4700,consumed4800,consumed4900,consumed5000,survey_id
0,100001,1,75,4,594.80627,Female,1,0,0,0,...,Yes,No,No,No,Yes,Yes,Yes,Yes,No,100000
1,100002,1,150,4,1676.27230,Female,2,0,0,0,...,Yes,No,No,No,No,Yes,Yes,No,No,100000
2,100003,1,375,4,506.93719,Male,5,0,0,2,...,Yes,Yes,No,Yes,Yes,Yes,Yes,No,Yes,100000
3,100004,1,375,4,824.61786,Male,5,0,0,1,...,No,Yes,No,No,No,Yes,Yes,No,No,100000
4,100005,1,525,4,351.47644,Male,7,1,0,0,...,Yes,No,No,Yes,No,Yes,Yes,Yes,No,100000


In [43]:
# Check dimensions of training targets
print("Shape:", hh_data.shape)  # (rows, columns)
print("Rows:", len(hh_data))
print("Columns:", len(hh_data.columns))
hh_data.head()

Shape: (104234, 3)
Rows: 104234
Columns: 3


,survey_id,hhid,cons_ppp17
0,100000,100001,25.258402
1,100000,100002,16.996706
2,100000,100003,13.671848
3,100000,100004,7.189475
4,100000,100005,12.308855


In [44]:
# Check dimensions of poverty data
print("Shape:", pr_data.shape)  # (rows, columns)
print("Rows:", len(pr_data))
print("Columns:", len(pr_data.columns))
pr_data.head()

Shape: (3, 20)
Rows: 3
Columns: 20


,survey_id,pct_hh_below_3.17,pct_hh_below_3.94,pct_hh_below_4.60,pct_hh_below_5.26,pct_hh_below_5.88,pct_hh_below_6.47,pct_hh_below_7.06,pct_hh_below_7.70,pct_hh_below_8.40,pct_hh_below_9.13,pct_hh_below_9.87,pct_hh_below_10.70,pct_hh_below_11.62,pct_hh_below_12.69,pct_hh_below_14.03,pct_hh_below_15.64,pct_hh_below_17.76,pct_hh_below_20.99,pct_hh_below_27.37
0,100000,0.067364,0.118927,0.169905,0.221865,0.271564,0.319585,0.366329,0.419816,0.471454,0.523798,0.574413,0.623091,0.671263,0.721329,0.773303,0.819770,0.865121,0.909075,0.954239
1,200000,0.059326,0.111560,0.159023,0.211754,0.263100,0.311758,0.356914,0.407631,0.463443,0.512931,0.559361,0.609337,0.659291,0.708043,0.760932,0.809045,0.860350,0.906385,0.952805
2,300000,0.049803,0.100381,0.149502,0.200144,0.250192,0.300211,0.349596,0.399930,0.449845,0.499930,0.550082,0.599926,0.650088,0.699617,0.750341,0.800111,0.850081,0.899974,0.949988


## Proverty Rate Calculation

In [46]:
hhs_data = hh_data[['survey_id','cons_ppp17']]
w_data = train_data[['survey_id','weight']]

In [47]:
def get_poverty_thresholds(values, weights, surveys, pr_data):

    # Define thresholds
    thresholds = [3.17, 3.94, 4.60, 5.26, 5.88, 6.47, 7.06, 7.70, 8.40, 9.13, 9.87, 10.70, 11.62, 12.69, 14.03, 15.64, 17.76, 20.99, 27.37]
    thresholds = np.array(thresholds, dtype=float)
    pr = []

    for s in range(surveys.shape[0]):
        survey_values = values[values['survey_id'] == surveys[s]][['cons_ppp17']].to_numpy().flatten()
        survey_weights = weights[weights['survey_id'] == surveys[s]][['weight']].to_numpy().flatten()

        # Calculate percentage below each 
        percentages = [(survey_weights[survey_values < t].sum() / survey_weights.sum()) for t in thresholds]
        pr.append(percentages)

    df = pd.DataFrame([pr[0]], columns=pr_data.columns.tolist()[1:])
    for d in range(1,len(pr)):
        df = pd.concat([df, pd.DataFrame([pr[d]], columns=pr_data.columns.tolist()[1:])], ignore_index=True)
    
    df.index = surveys
    df.index.name = "survey_id"

    return df

df = get_poverty_thresholds(hh_data, w_data, surveys=hh_data["survey_id"].unique(), pr_data=pr_data)

In [49]:
df.head()

,pct_hh_below_3.17,pct_hh_below_3.94,pct_hh_below_4.60,pct_hh_below_5.26,pct_hh_below_5.88,pct_hh_below_6.47,pct_hh_below_7.06,pct_hh_below_7.70,pct_hh_below_8.40,pct_hh_below_9.13,pct_hh_below_9.87,pct_hh_below_10.70,pct_hh_below_11.62,pct_hh_below_12.69,pct_hh_below_14.03,pct_hh_below_15.64,pct_hh_below_17.76,pct_hh_below_20.99,pct_hh_below_27.37
survey_id,,,,,,,,,,,,,,,,,,,
100000,0.067364,0.118927,0.169905,0.221865,0.271564,0.319585,0.366329,0.419816,0.471454,0.523798,0.574413,0.623091,0.671263,0.721329,0.773303,0.819770,0.865121,0.909075,0.954239
200000,0.059326,0.111560,0.159023,0.211754,0.263100,0.311758,0.356914,0.407631,0.463443,0.512931,0.559361,0.609337,0.659291,0.708043,0.760932,0.809045,0.860350,0.906385,0.952805
300000,0.049803,0.100381,0.149502,0.200144,0.250192,0.300211,0.349596,0.399930,0.449845,0.499930,0.550082,0.599926,0.650088,0.699617,0.750341,0.800111,0.850081,0.899974,0.949988


## Check NaNs

In [50]:
# Calculate percentage of missing values in each column
nan_percent = train_data.isna().mean() * 100

In [51]:
# Remove variables with more than 10% missing values
missing_percent = nan_percent[nan_percent > 10]
print("Variables with more than 10% missing values:")
print(missing_percent)

# Remove these variables from the dataset
train_data = train_data.drop(columns=missing_percent.index.tolist())

Variables with more than 10% missing values:
sector1d    13.555078
dtype: float64


## Remove Variables Not Used As Features

In [52]:
# Remove columns from train_data
weights = train_data['weight']
survey = train_data['survey_id']
columns_to_remove = ['hhid', 'com', 'strata', 'weight']
train_data = train_data.drop(columns=columns_to_remove)

In [39]:
train_data.head()

,utl_exp_ppp17,male,hsize,num_children5,num_children10,num_children18,age,owner,water,toilet,...,consumed4200,consumed4300,consumed4400,consumed4500,consumed4600,consumed4700,consumed4800,consumed4900,consumed5000,survey_id
0,594.80627,Female,1,0,0,0,75,Owner,Access,Access,...,Yes,No,No,No,Yes,Yes,Yes,Yes,No,100000
1,1676.27230,Female,2,0,0,0,61,Owner,Access,Access,...,Yes,No,No,No,No,Yes,Yes,No,No,100000
2,506.93719,Male,5,0,0,2,49,Owner,Access,Access,...,Yes,Yes,No,Yes,Yes,Yes,Yes,No,Yes,100000
3,824.61786,Male,5,0,0,1,58,Not owner,Access,Access,...,No,Yes,No,No,No,Yes,Yes,No,No,100000
4,351.47644,Male,7,1,0,0,57,Owner,Access,Access,...,Yes,No,No,Yes,No,Yes,Yes,Yes,No,100000


## Target Variable

In [54]:
y_train = hh_data["cons_ppp17"]

## Create Pipeline

### Infer Feature Types

In [58]:

def infer_feature_types(df, year_col="year"):
    feature_cols = [c for c in df.columns]
    # Simple heuristic: non-numeric or low-cardinality numeric treated as categorical (including year)
    numeric_cols = [c for c in feature_cols if pd.api.types.is_numeric_dtype(df[c])]
    categorical_cols = [c for c in feature_cols if c not in numeric_cols]

    # Ensure year is treated as categorical (common for pooled models)
    if year_col in numeric_cols:
        numeric_cols.remove(year_col)
    if year_col not in categorical_cols:
        categorical_cols.append(year_col)
    return numeric_cols, categorical_cols

numeric_features, categorical_features = infer_feature_types(train_data, year_col="survey_id")
print("Numeric features:", numeric_features)
print("Categorical features:", categorical_features)


Numeric features: ['utl_exp_ppp17', 'hsize', 'num_children5', 'num_children10', 'num_children18', 'age', 'num_adult_female', 'num_adult_male', 'num_elderly', 'sworkershh', 'share_secondary', 'sfworkershh', 'region1', 'region2', 'region3', 'region4', 'region5', 'region6', 'region7']
Categorical features: ['male', 'owner', 'water', 'toilet', 'sewer', 'elect', 'water_source', 'sanitation_source', 'dweltyp', 'employed', 'educ_max', 'any_nonagric', 'urban', 'consumed100', 'consumed200', 'consumed300', 'consumed400', 'consumed500', 'consumed600', 'consumed700', 'consumed800', 'consumed900', 'consumed1000', 'consumed1100', 'consumed1200', 'consumed1300', 'consumed1400', 'consumed1500', 'consumed1600', 'consumed1700', 'consumed1800', 'consumed1900', 'consumed2000', 'consumed2100', 'consumed2200', 'consumed2300', 'consumed2400', 'consumed2500', 'consumed2600', 'consumed2700', 'consumed2800', 'consumed2900', 'consumed3000', 'consumed3100', 'consumed3200', 'consumed3300', 'consumed3400', 'consume

    ### Pipeline

In [60]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Preprocess: impute NaNs, then encode/scale as needed
preprocess = ColumnTransformer(
    transformers=[
        # Median imputation for numeric, then (optional) scale
        ("num", Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler(with_mean=True, with_std=True)),
        ]), numeric_features),

        # Most-frequent imputation for categorical, then one-hot encode
        ("cat", Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("ohe", OneHotEncoder(handle_unknown="ignore")),
        ]), categorical_features),
    ],
    remainder="drop",
)

### Model

In [ ]:
model = RandomForestRegressor(random_state=42,n_jobs=-1)

In [ ]:

def pooled_regression(df, target_col="y", year_col="year",
                      model=None, test_size=0.2, random_state=42):
    # Default model: Random Forest (robust, handles nonlinearity)
    if model is None:
        model = RandomForestRegressor(
            n_estimators=400,
            max_depth=None,
            random_state=random_state,
            n_jobs=-1
        )

    df_obs, df_mis = split_observed_missing(df, target_col)
    numeric_cols, categorical_cols = infer_feature_types(df, target_col, year_col)

    # Preprocess: scale numeric (optional), one-hot categorical (including 'year')
    preproc = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(with_mean=True, with_std=True), numeric_cols),
            ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ],
        remainder="drop"
    )

    pipe = Pipeline(steps=[("prep", preproc), ("model", model)])

    # Train/validation split only on observed rows
    X = df_obs.drop(columns=[target_col])
    y = df_obs[target_col]
    X_tr, X_te, y_tr, y_te = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    pipe.fit(X_tr, y_tr)

    # Evaluate (on observed years)
    y_pred_tr = pipe.predict(X_tr)
    y_pred_te = pipe.predict(X_te)

    metrics = {
        "train_R2": r2_score(y_tr, y_pred_tr),
        "test_R2": r2_score(y_te, y_pred_te),
        "train_RMSE": mean_squared_error(y_tr, y_pred_tr, squared=False),
        "test_RMSE": mean_squared_error(y_te, y_pred_te, squared=False),
    }

    # Predict missing years (impute)
    if not df_mis.empty:
        X_missing = df_mis.drop(columns=[target_col])
        y_imputed = pipe.predict(X_missing)
        df_imputed = df_mis.copy()
        df_imputed[target_col] = y_imputed
        # Combine back with observed to get a “completed” dataset
        df_completed = pd.concat([df_obs, df_imputed], axis=0).sort_index()
    else:
        df_completed = df.copy()

    return pipe, metrics, df_completed



In [ ]:

# Example: df has columns ['year', 'age', 'education', 'region', 'y']
# where 'y' is continuous (e.g., income) and is missing in some years
model, metrics, df_completed = pooled_regression(df, target_col="y", year_col="year")
print(metrics)
# df_completed now contains imputed y for the years where it was missing


In [ ]:

import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error


def train_and_impute_across_years(
    df: pd.DataFrame,
    target_col: str = "cons_pc",
    year_col: str = "year",
    train_years=None,
    test_years=None,
    model=None,
):
    """
    Train a pooled model using multiple years and predict the target for test years.
    
    Parameters
    ----------
    df : DataFrame
        Tidy data with columns: `year_col`, `target_col`, and feature columns.
    target_col : str
        Name of the target variable (e.g., per-capita consumption).
    year_col : str
        Name of the year column.
    train_years : list or array-like
        Years to use for training. If None, uses all years except `test_years`.
    test_years : list or array-like
        Years to predict. If None, uses the most recent year as test.
    model : sklearn regressor or None
        If None, uses RandomForestRegressor with reasonable defaults.

    Returns
    -------
    pipe : fitted sklearn Pipeline
    metrics_by_year : dict
        Dictionary of metrics (R2, RMSE) computed on test years where target is available.
    df_with_preds : DataFrame
        Input DataFrame with a new column f'{target_col}_pred' for rows in test_years.
    """
    df = df.copy()

    # Decide train/test years if not provided
    all_years = sorted(pd.unique(df[year_col]))
    if test_years is None:
        test_years = [max(all_years)]  # last year as default test
    if train_years is None:
        train_years = [y for y in all_years if y not in set(test_years)]

    # Split train/test rows
    is_train = df[year_col].isin(train_years) & df[target_col].notna()
    is_test  = df[year_col].isin(test_years)

    if is_train.sum() == 0:
        raise ValueError("No training rows found. Check `train_years` and presence of target.")
    if is_test.sum() == 0:
        raise ValueError("No test rows found. Check `test_years`.")

    # Feature columns: everything except target
    feature_cols = [c for c in df.columns if c != target_col]

    # Infer numeric/categorical features
    numeric_cols = [c for c in feature_cols if pd.api.types.is_numeric_dtype(df[c]) and c != year_col]
    categorical_cols = [c for c in feature_cols if c not in numeric_cols]

    # Ensure year is treated as categorical (pooled‑years approach)
    if year_col in numeric_cols:
        numeric_cols.remove(year_col)
    if year_col not in categorical_cols:
        categorical_cols.append(year_col)

    preproc = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), numeric_cols),
            ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ],
        remainder="drop",
    )

    if model is None:
        model = RandomForestRegressor(
            n_estimators=400,
            random_state=42,
            n_jobs=-1
        )

    pipe = Pipeline(steps=[("prep", preproc), ("model", model)])

    # Fit on training years (only rows with target)
    X_train = df.loc=is_train, feature_cols]
    y_train = df.loc=is_train, target_col]

    pipe.fit(X_train, y_train)

    # Predict on test years
    X_test = df.loc=is_test, feature_cols]
    y_pred = pipe.predict(X_test)

    pred_col = f"{target_col}_pred"
    df.loc[is_test, pred_col] = y_pred

    # Evaluate per test year where the true target is available (optional)
    metrics_by_year = {}
    for y in test_years:
        mask = (df[year_col] == y) & df[target_col].notna() & df[pred_col].notna()
        if mask.sum() > 1:  # need at least 2 points for R2
            y_true = df.loc[mask, target_col]
            y_hat  = df.loc[mask, pred_col]
            r2 = r2_score(y_true, y_hat)
            rmse = mean_squared_error(y_true, y_hat, squared=False)
            metrics_by_year[y] = {"R2": r2, "RMSE": rmse}
        else:
            # No ground truth available in that year (e.g., true imputation year)
            metrics_by_year[y] = {"R2": np.nan, "RMSE": np.nan}

    return pipe, metrics_by_year, df


# --------------------------
# Example usage
# --------------------------
if __name__ == "__main__":
    # Example: df has columns ['year','region','urban','hh_size','educ_head','cons_pc', ...]
    # Replace with your actual DataFrame.
    # df = pd.read_csv("your_survey_data.csv")
    # For demonstration, create a small dummy dataset:
    rng = np.random.default_rng(0)
    n = 1000
    df_demo = pd.DataFrame({
        "year": rng.choice([2017, 2018, 2019, 2020, 2021], size=n),
        "region": rng.choice(["N", "S", "E", "W"], size=n),
        "urban": rng.choice([0, 1], size=n),
        "hh_size": rng.integers(1, 8, size=n),
        "educ_head": rng.choice(["none", "primary", "secondary", "tertiary"], size=n),
        "cons_pc": rng.normal(1000, 250, size=n).clip(200, None),
    })

    # Suppose we want to train on 2017–2020 and predict 2021 (test/imputation)
    pipe, metrics, df_out = train_and_impute_across_years(
        df_demo,
        target_col="cons_pc",
        year_col="year",
        train_years=[2017, 2018, 2019, 2020],
        test_years=[2021]
    )

    print("Evaluation on test years (if ground truth available):")
    print(metrics)

    # df_out now contains 'cons_pc_pred' for rows in test years
    print(df_out.head())
